Figure 2E 재현하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 76 (delta 26), reused 59 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 318.16 KiB | 5.78 MiB/s, done.
Resolving deltas: 100% (26/26), done.
./
./root/
./root/.condarc
./root/.tmux.conf
./root/.profile
./root/.bashrc.biolab
./root/.bin.priority/
./root/.bin.priority/pip2
./root/.bin.priority/pip3
./root/.bin.priority/pip
./root/.vimrc
--2025-05-30 02:17:49--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155472915 (148M) [application/octet-stream]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>] 1

In [ ]:
# 필요한 패키지 설치 (최초 1회 실행)
!pip install pysam logomaker pybedtools

import os
import pysam
import pandas as pd
import numpy as np
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import logomaker
from collections import defaultdict
# pybedtools는 추후 annotation 등에 활용할 수 있으므로 설치해 둡니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 82.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 63.9 MB/s eta 0:00:00
  Created wheel for pybedtools: filename=pybedtools-0.12.0-cp311-cp311-linux_x86_64.whl size=14261426 sha256=fa882911baff6cc7dd08c96297befa8c1d98bfea18bd248a92198ada1b88daaf
  Stored in directory: /root/.cache/pip/wheels/95/62/57/e8c47c2a43884227b6644cf1d2dabf3162a0f40b9c1493c1f5
Successfully built pybedtools


In [ ]:
# =============================
# [Step 2] CLIP‑seq BAM 파일로부터 binding site 추출
# =============================
# 여기서는 CLIP-35L33G.bam 파일을 사용하여, genome 전반의 binding site 후보를
# pysam의 pileup 기능을 이용해 추출합니다.
bam_path = "/content/drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam"
bamfile = pysam.AlignmentFile(bam_path, "rb")

# coverage (read count) threshold를 설정합니다. (예, 10 이상인 포지션)
binding_sites = []
threshold = 10
for pileupcolumn in bamfile.pileup(max_depth=1000):
    if pileupcolumn.n >= threshold:
        binding_sites.append({
            "chrom": pileupcolumn.reference_name,
            "pos": pileupcolumn.pos + 1,  # pysam은 0-based, pos+1로 변환
            "count": pileupcolumn.n
        })
bamfile.close()
print("추출된 binding site 수:", len(binding_sites))
binding_df = pd.DataFrame(binding_sites)

In [ ]:

# =============================
# [Step 3] Reference Genome에서 binding site 주변 31 nt 시퀀스 추출
# =============================
# (주의: 여기서는 reference FASTA 파일이 필요합니다. 예를 들어 mm9.fa가 존재한다고 가정)
ref_fasta_path = "/content/drive/MyDrive/binfo1-datapack1/mm9.fa"  # 여러분의 파일명으로 수정하세요
fasta = pysam.FastaFile(ref_fasta_path)

# 각 binding site의 중심(1-based pos)을 기준으로 좌우 15 nt씩 총 31 nt 추출
def extract_sequence(chrom, pos, length=31):
    center_index = pos - 1  # 0-based
    start = center_index - (length // 2)
    end = center_index + (length // 2) + 1  # end는 non-inclusive
    seq = fasta.fetch(chrom, start, end).upper()
    return seq if len(seq) == length else None

binding_df["sequence"] = binding_df.apply(lambda row: extract_sequence(row["chrom"], row["pos"], 31), axis=1)
binding_df = binding_df.dropna(subset=["sequence"]).reset_index(drop=True)
print("Binding site DataFrame shape (31 nt sequences):", binding_df.shape)

# =============================
# [Step 4] 간단한 motif 분류
# =============================
# LIN28A가 인식하는 모티프는 여러 형태를 보일 수 있으므로,
# 여기서는 예시로 시퀀스 내에서 "AAGGAG"가 있으면 'AAGNNG' 그룹, "AAGAG"가 있으면 'AAGNG' 그룹, 그 외는 'Other'로 분류합니다.
def classify_motif(seq):
    if "AAGGAG" in seq:
        return "AAGNNG"
    elif "AAGAG" in seq:
        return "AAGNG"
    else:
        return "Other"

binding_df["motif_group"] = binding_df["sequence"].apply(classify_motif)
print(binding_df["motif_group"].value_counts())

# (원한다면, 추출된 binding site 정보를 CSV 파일로 저장할 수 있음)
# binding_df.to_csv("/content/drive/MyDrive/binfo1-datapack1/binding_sites_extracted.csv", index=False)

# =============================
# [Step 5] RNAfold를 이용해 각 binding site 시퀀스의 secondary structure 예측 및 WC pair 계산
# =============================

# RNAfold 실행 함수 (RNAfold가 Colab에 설치되어 있어야 합니다.)
def predict_structure(sequence):
    proc = subprocess.Popen(["RNAfold", "--noPS"],
                            stdin=subprocess.PIPE,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            text=True)
    stdout, stderr = proc.communicate(sequence + "\n")
    if proc.returncode != 0:
        raise RuntimeError("RNAfold error: " + stderr)
    lines = stdout.strip().split("\n")
    if len(lines) < 2:
        raise RuntimeError("Unexpected RNAfold output: " + stdout)
    # 두 번째 줄의 첫 번째 필드가 dot‑bracket 구조
    structure = lines[1].split()[0]
    return structure

# dot-bracket 구조에서 괄호 매칭을 통해 WC 쌍 추출 함수
def extract_wc_pairs(sequence, structure):
    stack = []
    pairs = {}
    for i, char in enumerate(structure):
        if char == "(":
            stack.append(i)
        elif char == ")":
            if stack:
                j = stack.pop()
                pairs[i] = j
                pairs[j] = i
    canonical = {("A", "U"), ("U", "A"), ("G", "C"), ("C", "G")}
    wc_pairs = []
    for i, j in pairs.items():
        if i < j and (sequence[i], sequence[j]) in canonical:
            wc_pairs.append((i, j))
    return wc_pairs

# 각 binding site 시퀀스에서 중심(인덱스 15)을 기준으로 WC pair의 상대 좌표 계산
def process_binding_site(seq):
    try:
        structure = predict_structure(seq)
    except Exception as e:
        structure = "." * len(seq)
    pairs = extract_wc_pairs(seq, structure)
    counts = defaultdict(int)
    center = 15  # 0-based 중심, 31 nt의 중앙
    for i, j in pairs:
        rel_i = i - center
        rel_j = j - center
        key = (min(rel_i, rel_j), max(rel_i, rel_j))
        counts[key] += 1
    return counts

# 모든 binding site에 대해 누적 WC pair 카운트 행렬 생성 (관측값)
def aggregate_counts(df):
    coords = range(-15, 16)
    Obs = pd.DataFrame(0, index=coords, columns=coords, dtype=float)
    for seq in df["sequence"]:
        if len(seq) != 31:
            continue
        counts = process_binding_site(seq)
        for (ri, rj), cnt in counts.items():
            Obs.at[ri, rj] += cnt
            if ri != rj:
                Obs.at[rj, ri] += cnt
    return Obs

# =============================
# [Step 6] 무작위 셔플을 통한 Background WC pair 행렬 계산
# =============================
def compute_background(df, n_iter=50):
    coords = range(-15, 16)
    Bg = pd.DataFrame(0, index=coords, columns=coords, dtype=float)
    total = 0
    for seq in df["sequence"]:
        if len(seq) != 31:
            continue
        for _ in range(n_iter):
            seq_list = list(seq)
            np.random.shuffle(seq_list)
            shuffled_seq = "".join(seq_list)
            try:
                structure = predict_structure(shuffled_seq)
            except Exception:
                structure = "." * len(shuffled_seq)
            pairs = extract_wc_pairs(shuffled_seq, structure)
            for i, j in pairs:
                rel_i = i - 15
                rel_j = j - 15
                key = (min(rel_i, rel_j), max(rel_i, rel_j))
                Bg.at[key[0], key[1]] += 1
                if key[0] != key[1]:
                    Bg.at[key[1], key[0]] += 1
            total += 1
    if total > 0:
        Bg = Bg / total
    return Bg

# =============================
# [Step 7] 모티프 그룹별로 binding site 분리 및 관측/배경 행렬 계산
# =============================
# 여기서는 예시로 'AAGNNG'와 'AAGNG' 그룹으로 나눕니다.
group1_df = binding_df[binding_df["motif_group"]=="AAGNNG"]
group2_df = binding_df[binding_df["motif_group"]=="AAGNG"]

Obs_matrix1 = aggregate_counts(group1_df)
Obs_matrix2 = aggregate_counts(group2_df)
Bg_matrix1 = compute_background(group1_df, n_iter=50)
Bg_matrix2 = compute_background(group2_df, n_iter=50)

# 정규화: (observed + small epsilon) / (background + epsilon) 후 log2 transform
eps = 1e-6
Norm1 = (Obs_matrix1 + eps) / (Bg_matrix1 + eps)
Norm2 = (Obs_matrix2 + eps) / (Bg_matrix2 + eps)
LogNorm1 = np.log2(Norm1)
LogNorm2 = np.log2(Norm2)

# =============================
# [Step 8] 시퀀스 로고 생성: 각 binding site에서 각 위치의 nucleotide 분포 계산
# =============================
def create_logo_dataframe(df):
    letters = ["A", "C", "G", "U"]
    counts = {pos: {l: 0 for l in letters} for pos in range(31)}
    for seq in df["sequence"]:
        if len(seq) != 31:
            continue
        for i, base in enumerate(seq):
            if base in letters:
                counts[i][base] += 1
    logo_df = pd.DataFrame(counts).T
    logo_df = logo_df.div(logo_df.sum(axis=1), axis=0)
    # 인덱스: 0 ~ 30 → 상대 좌표 -15 ~ +15
    logo_df.index = [i-15 for i in logo_df.index]
    return logo_df

Logo_df1 = create_logo_dataframe(group1_df)
Logo_df2 = create_logo_dataframe(group2_df)

# =============================
# [Step 9] 최종 시각화: 그룹별 시퀀스 로고와 정규화된 WC pair heatmap 생성
# =============================
plt.figure(figsize=(12,5))
ax_logo1 = plt.subplot(1,2,1)
ax_logo1.set_title("Sequence Logo (AAGNNG)")
logomaker.Logo(Logo_df1.fillna(0), ax=ax_logo1, shade_below=.5, fade_below=.5)
ax_logo1.set_xticks([])

ax_heat1 = plt.subplot(1,2,2)
sns.heatmap(LogNorm1, cmap="vlag", center=0, square=True,
            xticklabels=True, yticklabels=True,
            cbar_kws={'label': 'log2(observed/background)'}, ax=ax_heat1)
ax_heat1.set_title("Normalized WC Pair Heatmap (AAGNNG)")
ax_heat1.set_xlabel("Relative Position")
ax_heat1.set_ylabel("Relative Position")
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,5))
ax_logo2 = plt.subplot(1,2,1)
ax_logo2.set_title("Sequence Logo (AAGNG)")
logomaker.Logo(Logo_df2.fillna(0), ax=ax_logo2, shade_below=.5, fade_below=.5)
ax_logo2.set_xticks([])

ax_heat2 = plt.subplot(1,2,2)
sns.heatmap(LogNorm2, cmap="vlag", center=0, square=True,
            xticklabels=True, yticklabels=True,
            cbar_kws={'label': 'log2(observed/background)'}, ax=ax_heat2)
ax_heat2.set_title("Normalized WC Pair Heatmap (AAGNG)")
ax_heat2.set_xlabel("Relative Position")
ax_heat2.set_ylabel("Relative Position")
plt.tight_layout()
plt.show()
